# Resource Optimziation with Load Coupling in Multi-Cell NOMA
by [Lei You](https://leiyou.me/)

In [8]:
import numpy as np
import pandas as pd
import scipy.io
import math
import itertools
from sklearn.metrics import mean_absolute_error
import networkx as nx
from time import time

## Parameter configuration

In [9]:
MAX_ITER = 1000

In [10]:
gain_mat_dbm_ = scipy.io.loadmat('Gain_19-0-30.mat')['Expression1']

In [11]:
N_CELL, N_UE = gain_mat_dbm_.shape
print(N_CELL, N_UE)

N_MACRO = 19
N_SMALL = N_CELL - N_MACRO
gain_mat_dbm = gain_mat_dbm_.copy()

# power per RU in mw
p_macro_ru = 800 
p_small_ru = 0
power = np.array([p_macro_ru for i in range(N_MACRO)] + [p_small_ru for i in range(N_SMALL)])


19 570


In [12]:
# N_CELL, N_UE = 1,30

# gain_mat_dbm = gain_mat_dbm_[:N_CELL, :N_UE].copy()

# p_macro_ru = 200
# power = np.array([p_macro_ru for i in range(N_CELL)])

In [13]:
cell_list = range(N_CELL)
ue_list = range(N_UE)

In [14]:
def dbm_to_mw(x):
    return 10**(x/10)

def mw_to_dbm(x):
    return 10*math.log10(x)

In [15]:
# tolerance for bisection/golden-section convergence
epsilon = 10**(-8) 

# sigma
sigma = dbm_to_mw(-174)

# bandwidth on one RU
bandwidth = 180000

# noise
noise = sigma * bandwidth

# convert gain matrix from dbm to mw
gain_mat = np.array(list(map(dbm_to_mw, gain_mat_dbm)))

# normalized demand
demand= 0.074571
demand_list = np.array([demand for j in range(N_UE)])

In [16]:
# cell-ue association
association = np.zeros((N_CELL, N_UE))
for j in ue_list:
    c = 0
    signal_max = 0
    for i in cell_list:
        signal_ij = power[i]*gain_mat[i][j]
        if signal_ij >= signal_max:
            signal_max = signal_ij
            c = i
    association[c][j] = 1

In [17]:
# cell's candidate UEs
ue_list_of_cells = []
for i in cell_list:
    ue_list_cell_i = []
    for j in ue_list:
        if association[i][j]==1:
            ue_list_cell_i.append(j)
    ue_list_of_cells.append(ue_list_cell_i)
    
# cell's candidate pairs
pair_list_of_cells = []
for i in cell_list:
    pair_list_cell_i = list(itertools.combinations(ue_list_of_cells[i],2))
    pair_list_of_cells.append(pair_list_cell_i)
    
# UE's serving cell
cell_of_ues = [np.where(association[:,ue] == 1)[0][0] for ue in ue_list ]       
scipy.io.savemat('./cell_of_ues.mat', {'association':cell_of_ues})

In [18]:
# initial load
init_load = np.zeros(N_CELL)

## Program of OMA

In [20]:
def w(j, load):
    i = cell_of_ues[j]
    gain_ij = gain_mat[i][j]
    inter_cell_interf_j = np.dot(power*load, gain_mat[:,j]) - power[i]*load[i]*gain_ij
    w_j = (inter_cell_interf_j + noise) / gain_ij
    
    return w_j


def sinr_oma(j, load):
    i = cell_of_ues[j]
    w_j = w(j, load)
    return power[i] / w_j
            
    
def f_oma(load, demand_list):
    sinr_list = np.array([sinr_oma(j, load) for j in ue_list])
    capacity_list = np.array(list(map(lambda x: math.log2(1+x), sinr_list)))
    ue_load = demand_list/capacity_list
    return np.dot(association, ue_load)


def fixed_point_iteration(load, demand_list):
    for i in range(MAX_ITER):
        load_new = f_oma(load, demand_list)
        if mean_absolute_error(load, load_new) < epsilon:
            break
        else:
            load = load_new
    return load

In [21]:
oma_load = fixed_point_iteration(init_load, demand_list)
oma_load, max(oma_load)

(array([0.83954341, 0.77523053, 0.71402201, 0.99999769, 0.92779481,
        0.69044691, 0.71754195, 0.75165448, 0.77734492, 0.67650388,
        0.79514204, 0.74002605, 0.6988361 , 0.55188615, 0.80683691,
        0.94801512, 0.77146765, 0.64568017, 0.73453918]),
 0.9999976880691546)

## Program of NOMA

In [64]:
def bss(func, low=0, high=30):
    """Bi-section search
    'Find root of continuous function where f(low) and f(high) have opposite signs'
    """

    for i in range(MAX_ITER):
        midpoint = (low + high) / 2.0
        if func(low)*func(midpoint) > 0:
            low = midpoint
        else:
            high = midpoint
            
        if abs(func(midpoint))<epsilon:
            break

    return midpoint

def gss(f, a=0, b=1):
    """Golden section search
    to find the minimum of f on [a,b]
    f: a strictly unimodal function on [a,b]

    Example:
    >>> f = lambda x: (x-2)**2
    >>> x = gss(f, 1, 5)
    >>> print("%.15f" % x)
    2.000009644875678

    """
    gr = (math.sqrt(5) + 1) / 2
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(c - d) > epsilon:
        if f(c) < f(d):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr
    
    return (b + a) / 2

In [65]:
def good_bad_ues(pair, load):
    if w(pair[0], load) < w(pair[1], load):
        return int(pair[0]), int(pair[1])
    else:
        return int(pair[1]), int(pair[0])  


def sinr_noma(j, h, q_j, q_h, load):
    w_j = w(j, load)
    w_h = w(h, load)
    
    theta_hj = 1 if w_h<=w_j else 0
    if w_h==w_j:
        theta_hj = 1 if j<h else 0
    
    sinr = q_j / (q_h * theta_hj + w_j)
    return sinr


def function_to_apply_bss(c_noma_h, w_j, w_h, c_oma_j, c_oma_h, p_i):
    """This function is obtained by solving (21) in the paper. 
       Namely, replace c_{+u} by c_{-u}*c_{-}/c_{+}
       Remark that j = + and h = - in this function.
    """
    
    return (w_j* 2**((c_oma_h/c_oma_j+1) *c_noma_h) + (w_h-w_j)* 2**(c_noma_h)) - (p_i+w_h)


def Cv(c_noma_j, c_noma_h, w_j, w_h, p_i, load):
    """This is function (32) in the paper. 
       Remark that j = + and h = -
    """
    
    numerator = w_j*2**(c_noma_j + c_noma_h) + (w_h-w_j) * 2**c_noma_h
    denominator = p_i + w_h
        
    return math.log2(numerator/denominator)


def Hj(x_u, load, w_j, w_h, d_j, d_h, p_i, c_noma_j_K):
    """This is function (33) in the paper.
       Remark that j = + and h = -
    """
    if c_noma_j_K * x_u > d_j:
        return d_j / x_u
    else:
        numerator = (p_i+w_h) - (w_h-w_j)*2**(d_h/x_u)
        denominator = w_j * 2**(d_h/x_u)
        return math.log2(numerator/denominator)

    
def Hh(x_u, load, w_j, w_h, d_j, d_h, p_i, c_noma_h_K):
    """This is function (34) in the paper.
       Remark that j = + and h = -
    """
    
    if c_noma_h_K * x_u > d_h:
        return d_h / x_u
    else:
        numerator = (p_i + w_h)
        denominator = (w_j * 2**(d_j/x_u) + w_h - w_j)
        return math.log2(numerator/denominator)
    
    
def C(x_u, pair, load, demand_list):
    """This is function (23) in the paper, but it returns values for both users in the pair.
       The first returned value is for the good UE (who decodes). 
       The second for the bad UE (who being decoded).
       Remark that j = + and h = -
    """
    j, h = good_bad_ues(pair, load)
        
    i = cell_of_ues[j]
    
    c_oma_j = math.log2(1+sinr_oma(j,load))
    c_oma_h = math.log2(1+sinr_oma(h,load))
    
    d_j = demand_list[j]
    d_h = demand_list[h]
    
    w_j = w(j, load)
    w_h = w(h, load) 
    
    p_i = power[i]
    
    # using bi-section search to compute point K
    c_noma_h_K = bss(
        lambda x: function_to_apply_bss(x, w_j, w_h, c_oma_j, c_oma_h, p_i),
    )
    c_noma_j_K = (c_oma_h/c_oma_j) * c_noma_h_K
    x_u_K = min(d_j/c_noma_j_K, d_h/c_noma_h_K)
    
    # check which case we are in
    if x_u <= min(d_j/c_noma_j_K, d_h/c_noma_h_K):   
        # Case 1, point K is the optimum
        c_noma_j = c_noma_j_K
        c_noma_h = c_noma_h_K
    elif Cv(d_j/x_u, d_h/x_u,  w_j, w_h, p_i, load)>0:
        # Case 2
        c_noma_j = Hj(x_u, load, w_j, w_h, d_j, d_h, p_i, c_noma_j_K)
        c_noma_h = Hh(x_u, load, w_j, w_h, d_j, d_h, p_i, c_noma_h_K)
    else:
        c_noma_j = d_j / x_u
        c_noma_h = d_h / x_u
        
    return c_noma_j, c_noma_h
    
def oma_allocation(x_u, pair, load, demand_list): 
    """This solves the second term (i.e. the optimization problem) in (18), given x_u.
       The returned value is the optimized x_{+} and x_{-} in (18).
       Remark that j = + and h = -      
    """
    j, h = good_bad_ues(pair, load)
        
    d_j = demand_list[j]
    d_h = demand_list[h]
    c_oma_j = math.log2(1+sinr_oma(j,load))
    c_oma_h = math.log2(1+sinr_oma(h,load))
    
    c_noma_j, c_noma_h = C(x_u, pair, load, demand_list)
    
    x_j = max((d_j - c_noma_j*x_u) / c_oma_j,0)
    x_h = max((d_h - c_noma_h*x_u) / c_oma_h,0)
    return x_j, x_h
        

def Z(x_u, pair, load, demand_list):
    """This is (18) in the paper.
    """
    x_j, x_h = oma_allocation(x_u, pair, load, demand_list)   
    return x_u + x_j + x_h


def split(pair, load, demand_list):
    """The algorithm "SPLIT" in the paper.
       Remark that j = + and h = -
       The returned 5 values must satisfy:
           value 1: power split of pair[0]
           value 2: power split of pair[1]
           value 3: oma load of pair[0]
           value 4: oma load of pair[1]
           value 5: noma load of pair
    """
    j, h = good_bad_ues(pair, load)
    d_j = demand_list[j]
    d_h = demand_list[h]
    c_oma_j = math.log2(1+sinr_oma(j,load))
    c_oma_h = math.log2(1+sinr_oma(h,load))
    
    i = cell_of_ues[j]
    w_j = w(j, load)
    w_h = w(h, load)
    
    p_i = power[i]
    
    x_u_best = gss(lambda x: Z(x, pair, load, demand_list))

    c_noma_j_best, c_noma_h_best = C(x_u_best, pair, load, demand_list)
    x_j_best = max((d_j - c_noma_j_best * x_u_best)/c_oma_j, 0)
    x_h_best = max((d_h - c_noma_h_best * x_u_best)/c_oma_h, 0)
    
    q_j_best = (2**c_noma_j_best - 1) * w_j
    q_h_best = p_i - q_j_best
        
    if pair == good_bad_ues(pair, load):
        return q_j_best, q_h_best, x_j_best, x_h_best, x_u_best
    else:
        return q_h_best, q_j_best, x_h_best, x_j_best, x_u_best    



In [66]:
def s_cell(i, load, demand_list):
    """The algorithm "S-CELL" in the paper.
    """
    non_reversed_weights = dict()

    even_pairs_solution = dict()
    for pair in pair_list_of_cells[i]:
        even_pairs_solution.update({pair: split(pair, load, demand_list)})
        _, _, x_j, x_h, x_u = even_pairs_solution[pair]
        non_reversed_weight_u = x_j + x_h + x_u
        non_reversed_weights.update({pair:non_reversed_weight_u})
        
    odd_pairs_solution = dict()
    if len(ue_list_of_cells[i]) % 2 != 0:
        for j in ue_list_of_cells[i]:
            c_oma_j = math.log2(1+sinr_oma(j,load))
            pair = (j,N_UE)
            odd_pairs_solution.update({pair: demand_list[j]/c_oma_j})
            non_reversed_weights.update({pair: odd_pairs_solution[pair]})
            
    max_weight = max(non_reversed_weights.values())
    weights = dict()
    for pair in non_reversed_weights.keys():
        weights.update({pair:max_weight-non_reversed_weights[pair]})
                
    Graph = nx.Graph()
    for pair in weights.keys():
        Graph.add_edge(pair[0], pair[1], weight= weights[pair])
    selected_pairs_unsorted = list(nx.max_weight_matching(Graph))
    selected_pairs = list(map(lambda x: tuple(sorted(x)), selected_pairs_unsorted))
        
    pair_load_noma_best = dict()
    ue_load_oma_best = dict()
    pair_power_best = dict()
    
    for pair in selected_pairs:
        if pair[1]==N_UE:
            # the odd pair that doing oma
            ue_load_oma_best.update({pair[0]: odd_pairs_solution[pair]})
        else:
            q_j, q_h, x_j, x_h, x_u = even_pairs_solution[pair]
            pair_power_best.update({pair: (q_j, q_h)})
            ue_load_oma_best.update({pair[0]: x_j})
            ue_load_oma_best.update({pair[1]: x_h})
            pair_load_noma_best.update({pair: x_u})
    
    noma_pairs = list( set(selected_pairs).intersection(set(pair_list_of_cells[i])) )
    
    cell_load_i = sum(ue_load_oma_best.values()) + sum(pair_load_noma_best.values())
    return cell_load_i, pair_power_best, ue_load_oma_best, pair_load_noma_best, noma_pairs

output_template = """Cell {}: {} \t elapsed time {}
        \n Power Allocation
        {}
        \n Resource Allocation (OMA)
        {}
        \n Resource Allocation (NOMA)
        {}
        \n Pair Selection
        {}
    """

def f_noma(load, demand_list, verbose=False):    
    power_allocation = []
    resource_allocation_oma = []
    resource_allocation_noma = []
    pair_selection = []
    load_new = []
    
    for i in cell_list:
        t = time()
        (cell_load_i, pair_power_best, 
         ue_load_oma_best, pair_load_noma_best, 
         noma_pairs) = s_cell(i, load, demand_list)

        load_new.append(cell_load_i)
        power_allocation.append(pair_power_best)
        resource_allocation_oma.append(ue_load_oma_best)
        resource_allocation_noma.append(pair_load_noma_best)
        pair_selection.append(noma_pairs)      
        if verbose:
            print(output_template.format(i, cell_load_i, round(time() - t, 3),
                      power_allocation[i],
                      resource_allocation_oma[i],
                      resource_allocation_noma[i],
                      pair_selection[i],
                      )
                 )
    return load_new, power_allocation, resource_allocation_oma, resource_allocation_noma, pair_selection
    


def m_cell(load, demand_list, verbose = False):
    """The algorithm M-CELL in the paper
    """

        
    for k in range(MAX_ITER):
        T = time()
        load_new, power_allocation, resource_allocation_oma, resource_allocation_noma, pair_selection = f_noma(load, demand_list, verbose)
        error = mean_absolute_error(load_new, load)
        
        if verbose:
            print('Round {} \n Cell load vector:\n {} \n error: {} \n elapsed time {}'.format(k, load_new, error, round(time()-T, 3)) )
        if error < 10**(-4):
            break
        else:
            load = load_new 

    return load, power_allocation, resource_allocation_oma, resource_allocation_noma, pair_selection
    

In [223]:
(noma_load, 
 power_allocation, 
 resource_allocation_oma, 
 resource_allocation_noma, 
 pair_selection) = m_cell(init_load, demand_list, verbose=False)

In [224]:
pd.DataFrame({
    'cell': range(N_CELL),
    'noma_load': noma_load,
'power_allocation': power_allocation,
 'resource_allocation_oma': resource_allocation_oma,
 'resource_allocation_noma': resource_allocation_noma,
 'pair_selection': pair_selection,
}).to_csv('solution.csv', index=False)

In [233]:
result_load = pd.DataFrame({
    'oma_load': oma_load,
    'noma_load': noma_load,
}).sort_values(by=['oma_load']).reset_index()

result_load.index = np.arange(1, len(result_load) + 1)
result_load

,index,oma_load,noma_load
1,13,0.551886,0.383110
2,17,0.645680,0.440080
3,9,0.676504,0.462693
4,5,0.690447,0.469611
5,12,0.698836,0.482766
6,2,0.714022,0.486828
7,6,0.717542,0.494703
8,18,0.734539,0.500553
9,11,0.740026,0.506012
10,7,0.751654,0.511560


In [242]:
string = ''
for i, load in zip(result_load.index, result_load.oma_load):
    string_pair = '(' + str(i) + str(',') + str(load) + ')'
    string += ' ' + string_pair
print(string)

 (1,0.5518861459993285) (2,0.6456801704701509) (3,0.6765038792178337) (4,0.6904469122376065) (5,0.69883609666146) (6,0.7140220134439574) (7,0.7175419491828879) (8,0.734539176494171) (9,0.7400260501227063) (10,0.7516544758359655) (11,0.7714676520929664) (12,0.7752305327497773) (13,0.7773449197257659) (14,0.7951420393176793) (15,0.806836912680391) (16,0.8395434052240811) (17,0.9277948087169523) (18,0.948015122217504) (19,0.9999976880691546)


In [243]:
string = ''
for i, load in zip(result_load.index, result_load.noma_load):
    string_pair = '(' + str(i) + str(',') + str(load) + ')'
    string += ' ' + string_pair
print(string)

 (1,0.3831101100077303) (2,0.4400796300584208) (3,0.46269308927566954) (4,0.4696107789002239) (5,0.4827656334496884) (6,0.4868277144342972) (7,0.4947032922736718) (8,0.5005532965317314) (9,0.5060115235843202) (10,0.511560077301295) (11,0.5262871092337478) (12,0.5362236826548274) (13,0.5447604243315015) (14,0.540933229652313) (15,0.5557959259517387) (16,0.5822758280551295) (17,0.6534936481549145) (18,0.651100515117767) (19,0.6966427658475751)


## Demand scaling

In [33]:
def demand_scaling_oma(load, load_limit, base_demand_list):
    for i in range(MAX_ITER):
        load_oma = f_oma(load, base_demand_list) 
        load_new = load_oma/( max(load_oma)/load_limit )
        if mean_absolute_error(load, load_new) < epsilon:
            break
        else:
            load = load_new
    
    load_oma = f_oma(load, base_demand_list) 
    alpha = 1 / ( max(load_oma)/load_limit )
    
    return load, alpha*base_demand_list


def demand_scaling_noma(load, load_limit, base_demand_list):
    for i in range(MAX_ITER):
        load_noma, _, _, _, _ = f_noma(load, base_demand_list)
        load_new = load_noma/( max(load_noma)/load_limit )
        if mean_absolute_error(load, load_new) < epsilon:
            break
        else:
            load = load_new
    
    load_noma, _, _, _, _ = f_noma(load, base_demand_list)
    alpha = 1 / ( max(load_noma)/load_limit )
    
    return load, alpha*base_demand_list

In [67]:
oma_load, oma_demand_list = demand_scaling_oma(init_load, 1.0, demand_list)

In [68]:
oma_demand_list

array([0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385,
       0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385,
       0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385,
       0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385,
       0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385,
       0.40161385, 0.40161385, 0.40161385, 0.40161385, 0.40161385])

In [126]:
noma_load, noma_demand_list = demand_scaling_noma(init_load, 1.0, demand_list)

In [ ]:
result = pd.DataFrame()

for load_limit in [0.2, 0.4, 0.6, 0.8, 1.0]:
    _, oma_demand_list = demand_scaling_oma(init_load, load_limit, demand_list)
    oma_demand = np.array(oma_demand_list).mean()
    
    _, noma_demand_list = demand_scaling_noma(init_load, load_limit, demand_list)
    noma_demand = np.array(noma_demand_list).mean()
    
    result = result.append({
        'load_limit': load_limit,
        'oma_demand': oma_demand,
        'noma_demand': noma_demand,
    }, ignore_index=True)

In [140]:
result = result.drop(['demand_increase_by_noma'], axis=1)

In [141]:
result['gain_by_noma'] = (result['noma_demand']-result['oma_demand']) / result['oma_demand']

In [185]:
display(result)

,load_limit,noma_demand,oma_demand,gain_by_noma
0,0.2,0.036198,0.029328,0.234258
1,0.4,0.055491,0.045523,0.218967
2,0.6,0.069167,0.057408,0.204833
3,0.8,0.079688,0.066814,0.192680
4,1.0,0.088159,0.074571,0.182220


### Demand conversion

In [244]:
TOT_BANDWIDTH = 20e+6 # Hz
SPECTRAL_EFFICIENCY = 30/1024/1024 # Mbps/Hz

In [245]:
demand_factor = TOT_BANDWIDTH * SPECTRAL_EFFICIENCY 

In [246]:
result_bps = result.copy()
result_bps['oma_demand'] = result_bps['oma_demand']*demand_factor
result_bps['noma_demand'] = result_bps['noma_demand']*demand_factor

In [247]:
result_bps

,load_limit,noma_demand,oma_demand,gain_by_noma
0,0.2,20.712707,16.781510,0.234258
1,0.4,31.752343,26.048562,0.218967
2,0.6,39.577755,32.849166,0.204833
3,0.8,45.597739,38.231322,0.192680
4,1.0,50.445240,42.669915,0.182220


In [41]:
result_bps.to_csv('demand_scaling.csv', index=False)